In [39]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [40]:
!pip install transformers[sentencepiece] datasets spacy scipy networkx numpy sent2vec pyngrok pandas nltk

In [41]:
!pip install rouge_score evaluate

In [42]:
# import library yang dibutuhkan
import networkx as nx
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import TFPegasusForConditionalGeneration, PegasusTokenizerFast

import re
import unicodedata

import pandas as pd
from rouge_score import rouge_scorer
from openpyxl import Workbook
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**PERINGKASAN BAHASA INDONESIA**

In [43]:
# Load the tokenizer and model
tokenizer_bert_id = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_bert_id = BertModel.from_pretrained('bert-base-multilingual-cased')

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 95abadbe-073a-423a-83bb-61b10837f299)')' thrown while requesting HEAD https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: ae6ff672-1019-4095-ac49-976db861215f)')' thrown while requesting HEAD https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json


In [44]:
def vector_id(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_id.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_id(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [45]:
#1. PRE-PROCESSING
def preprocess_id(text):

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = sent_tokenize(text)

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):

    text = sentences[i]

    # Tokenisasi (memecah kalimat menjadi kata)
    words = word_tokenize(text.lower())

    # Membersihkan tanda baca
    words = [word for word in words if word.isalnum()]

    # Menghapus stopwords
    stop_words = set(stopwords.words("indonesian"))
    words = [word for word in words if word not in stop_words]

    # Melakukan lemmatisasi
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Menggabungkan kembali kata yang telah dibersihkan menjadi kalimat
    result = " ".join(lemmatized_words)

    filtered_sentences.append(result)
  return filtered_sentences

In [46]:
# 2. PERINGKASAN EKSTRAKTIF

def extractive_sum_id(filtered_sentences, sentences, n):
  # mengubah kalimat menjadi vektor
  vectors = vector_id(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / n)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]

  # hasil peringkasan ekstraktif

  return " ".join(summary)

In [47]:
# 3. PERINGKASAN ABSTRAKTIF

# membersihkan teks
def text_cleaning(input_string):
    lowercase = input_string.lower()
    remove_link = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)', '', lowercase).replace("&amp;","&")
    remove_bullet = "\n".join([T for T in remove_link.split('\n') if '•' not in T and "baca juga:" not in T])
    remove_accented = unicodedata.normalize('NFKD', remove_bullet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    remove_parentheses = re.sub("([\(\|]).*?([\)\|])", "\g<1>\g<2>", remove_accented)
    remove_punc = re.sub(r"[^\w\d.\s]+",' ', remove_parentheses)
    remove_num_dot = re.sub(r"(?<=\d)\.|\.(?=\d)|(?<=#)\.","", remove_punc)
    remove_extra_whitespace =  re.sub(r'^\s*|\s\s*', ' ', remove_num_dot).strip()
    return ".".join([s for s in remove_extra_whitespace.strip().split('.') if len(s.strip())>10]).replace("_","")

In [48]:
model_name2 = "thonyyy/pegasus_indonesian_base-finetune"
model2 = TFPegasusForConditionalGeneration.from_pretrained(model_name2)
tokenizer2 = PegasusTokenizerFast.from_pretrained(model_name2)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 26abe93e-51c3-4788-81d7-d6b0b5fbd0ca)')' thrown while requesting HEAD https://huggingface.co/thonyyy/pegasus_indonesian_base-finetune/resolve/main/config.json
All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

All the layers of TFPegasusForConditionalGeneration were initialized from the model checkpoint at thonyyy/pegasus_indonesian_base-finetune.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFPegasusForConditionalGeneration for predictions without further training.
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 28e3f395-80ca-411c-9db4-b2f79e482111)')' thrown while requesting HEAD https://huggingface.co/thonyyy/pegasus_indonesian_base-finetune/resolve/main/generation_config.json
'(ReadTimeout

In [49]:
def abstractive_sum_id(text):
  clean_text = text_cleaning(text)
  inputs = tokenizer2(clean_text, return_tensors = 'tf')
  summary_ids = model2.generate(**inputs, max_new_tokens = 256, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer2.batch_decode(summary_ids, skip_special_tokens=True)

  return summary[0]

In [50]:
# MERINGKAS TEKS BAHASA
def summy_id(text, n):

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  sentences = sent_tokenize(text)

  # 1. PRE-PROCESSING
  result = preprocess_id(text)

  # 2. PERINGKASAN EKSTRAKTIF
  summary1 = extractive_sum_id(result, sentences, n)

  # 3. PERINGKASAN ABSTRAKTIF
  summary = abstractive_sum_id(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary

In [51]:
text = """

Sampah selalu kita temukan mengotori lingkungan di sekitar kita. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.

Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Walaupun sampah jelas-jelas membuat lingkungan tidak nyaman tetapi anehnya kesadaran kita terhadap lingkungan masih jauh dari cukup.

Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk.

Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan.

"""

In [52]:
result_id = summy_id(text, 2)

In [53]:
print(result_id)

sampah bisa membuat suasana menjadi rusak seketika karena bau menyengat. sampah bisa membuat suasana menjadi rusak seketika karena bau menyengat


**PERINGKASAN BAHASA INDONESIA PENGUJIAN**

In [54]:
# 1. TEXTRANK
def f_textrank_id(text, n):
  sentences = sent_tokenize(text)

  result = preprocess_id(text)

  # melakukan peringkasan ekstraktif
  summary = extractive_sum_id(result, sentences, n)

  return summary

In [55]:
# 2. PEGASUS
def f_pegasus_id(text):
  summary = abstractive_sum_id(text)
  return summary

In [56]:
# 3. HYBRID
def f_hybrid_id(text, n):
  summary = summy_id(text, n)
  return summary

In [57]:
text2 = """

Sampah selalu kita temukan mengotori lingkungan di sekitar kita. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.

Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Walaupun sampah jelas-jelas membuat lingkungan tidak nyaman tetapi anehnya kesadaran kita terhadap lingkungan masih jauh dari cukup.

Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk.

Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan.

"""

In [58]:
hasil_textrank_id = f_textrank_id(text2, 3)
hasil_pegasus_id = f_pegasus_id(text2)
hasil_hybrid_id = f_hybrid_id(text2, 2)

In [59]:
hasil_textrank_id

'Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.'

In [60]:
hasil_pegasus_id

'sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. padahal sampah jelas jelas membuat lingkungan tidak nyaman tetapi anehnya membuat lingkungan masih jauh dari cukup banyak'

In [61]:
hasil_hybrid_id

'sampah bisa membuat suasana menjadi rusak seketika karena bau menyengat. sampah bisa membuat suasana menjadi rusak seketika karena bau menyengat'

**PENGUJIAN PADA MASING MASING DATASET**

In [62]:
# import library yang dibutuhkan
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import datasets
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
from tqdm import tqdm
from transformers import pipeline
from datasets import load_dataset, load_metric


In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
# global config

sns.set_style('ticks')

In [65]:
# Mendeteksi file didalam diretori indosum
DATASET_ROOT = '/content/drive/MyDrive/NLP/fine_tune2/indosum'

files_id_dir = os.listdir(DATASET_ROOT)
test_files = []

for filename in files_id_dir:
    if 'test.01' in filename:
        test_files.append(filename
        )

test_files

['test.01.jsonl']

In [66]:
# fungsi mengubah file menjadi json objek
def load_file_to_json_list(filename):
    file = os.path.join(DATASET_ROOT, filename)
    data = []
    with open(file, 'r') as f:
        json_list = list(f)
        for json_str in tqdm(json_list, desc=f'Loading data {filename}'):
            d = json.loads(json_str)
            data.append(d)
    return data


# mengubah label menjadi string dictionary
def label_to_dict_str(label_list):
    label_dict = {} # key = paragraph_id : value = label list
    for i, label in enumerate(label_list[:]):
        label_dict[i] = label

    json_str = json.dumps(label_dict)
    num = len(label_dict)
    return json_str, num

# mengubah paragraf menjadi list
def paragraph_to_dict_str(paragraph_list):
    paragraph_dict = {} # key = paragraph_id : value = paragraph list
    for i, paragraph in enumerate(paragraph_list):
        new_paragraph = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            new_paragraph.append(sentence)
        paragraph_dict[i] = new_paragraph

    json_str = json.dumps(paragraph_dict)
    num = len(paragraph_dict)
    return json_str, num

# mengubah paragraf menjadi text
def paragraph_to_text(raw_paragraph_list):
    new_paragraph_list = []
    for i, paragraph in enumerate(raw_paragraph_list):
        paragraph_list = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            paragraph_list.append(sentence)

        new_paragraph = ' '.join(paragraph_list)
        new_paragraph_list.append(new_paragraph)

    paragraph_str = ' '.join(new_paragraph_list)
    return paragraph_str

# mengubah summary menjadi string dictionary
def summary_to_dict_str(summary_list):
    summary_dict = {} # key = summary_id : value = summary sentence
    for i, summary in enumerate(summary_list):
        summary_dict[i] = ' '.join(summary)

    json_str = json.dumps(summary_dict)
    num = len(summary_dict)
    return json_str, num

def summary_to_text(raw_summary_list):
    summary_list = []
    for i, summary in enumerate(raw_summary_list):
        summary_list.append(' '.join(summary))

    summary_str = ' '.join(summary_list)
    return summary_str


# melakukan alter data json
def alter_json_data(json_list_data, filename=''):
    new_json_list = []
    for json_data in tqdm(json_list_data, desc=f'Altering json data {filename}'):
        json_data = json_data.copy()
        json_data['gold_labels'], _ = label_to_dict_str(json_data['gold_labels'])
        json_data['news_text'] = paragraph_to_text(json_data['paragraphs'])
        json_data['paragraphs'], num_paragraph = paragraph_to_dict_str(json_data['paragraphs'])
        json_data['num_of_paragraphs'] = num_paragraph
        json_data['summary_text'] = summary_to_text(json_data['summary'])
        json_data['summary'], num_summary = summary_to_dict_str(json_data['summary'])
        json_data['num_of_summary'] = num_summary

        new_json_list.append(json_data)

    return new_json_list

# membuat dataset
def create_dataset(jsonl):
    header = list(jsonl[0].keys())
    dataset_list = []
    for json_data in jsonl:
        row = []
        for h in header:
            row.append(json_data[h])
        dataset_list.append(row)

    return header, dataset_list


# membuat dataset dari file
def create_dataset_from_files(file_list):
    df_header = None
    dataset_list = []
    for filename in file_list:
        json_l = load_file_to_json_list(filename)
        new_json_l = alter_json_data(json_l, filename)
        header, dataset_part = create_dataset(new_json_l)

        if not df_header: df_header = header
        dataset_list.extend(dataset_part)

    df_full = pd.DataFrame().from_records(dataset_list)
    df_full = df_full.rename(columns=dict(enumerate(header)))
    return df_full



In [67]:
# konversi data file menjadi data frame
df_test = create_dataset_from_files(test_files)

Altering json data test.01.jsonl: 100%|██████████| 3762/3762 [00:00<00:00, 11960.29it/s]


In [68]:
# konversi dataframe menjadi hugging face dataset
data_indosum_test = df_test[['id', 'news_text', 'summary_text']].iloc[0:1000]

### konversi data test
test_dataset = ds.dataset(pa.Table.from_pandas(data_indosum_test).to_batches())
hg_test_dataset = Dataset(pa.Table.from_pandas(data_indosum_test))

dataset_bahasa = datasets.DatasetDict({"test":hg_test_dataset})

In [69]:
dataset_bahasa.column_names

{'test': ['id', 'news_text', 'summary_text']}

In [70]:
dataset_bahasa['test']

Dataset({
    features: ['id', 'news_text', 'summary_text'],
    num_rows: 1000
})

MELAKUKAN PENGUJIAN PADA 100 ARTIKEL INDOSUM

In [71]:
import evaluate

In [72]:
%cd /content/drive/MyDrive/NLP/f_zero/hasil_uji

/content/drive/MyDrive/NLP/f_zero/hasil_uji


In [73]:
def save_test_data(file_name, reference_summary, generated_summary):
  # Inisialisasi ROUGE scorer
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  # Menghitung skor ROUGE untuk setiap pasangan referensi dan hasil yang dihasilkan
  scores = []
  for ref, gen in zip(reference_summary, generated_summary):
      score = scorer.score(ref, gen)
      scores.append(score)

  # Membuat DataFrame dari skor ROUGE, precision, dan recall
  data = {
      'Reference': reference_summary,
      'Generated': generated_summary,
      'ROUGE-1 Precision': [score['rouge1'].precision for score in scores],
      'ROUGE-1 Recall': [score['rouge1'].recall for score in scores],
      'ROUGE-1 F1-score': [score['rouge1'].fmeasure for score in scores],
      'ROUGE-2 Precision': [score['rouge2'].precision for score in scores],
      'ROUGE-2 Recall': [score['rouge2'].recall for score in scores],
      'ROUGE-2 F1-score': [score['rouge2'].fmeasure for score in scores],
      'ROUGE-L Precision': [score['rougeL'].precision for score in scores],
      'ROUGE-L Recall': [score['rougeL'].recall for score in scores],
      'ROUGE-L F1-score': [score['rougeL'].fmeasure for score in scores],
  }

  df = pd.DataFrame(data)

  # Menyimpan DataFrame ke file Excel
  output_excel = file_name
  df.to_excel(output_excel, index=False, engine='openpyxl')

  return(f"DataFrame saved to {output_excel}")


In [74]:
summary_textrank_bahasa = []
summary_pegasus_bahasa = []
summary_hybrid_bahasa = []
reference_summary_bahasa = []

limit = 100

In [75]:
for i in range (limit):
  # Inisial reference
  reference = dataset_bahasa["test"][i]["summary_text"]
  reference_summary_bahasa.append(reference)

In [76]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_bahasa["test"][i]["news_text"]
  result = f_textrank_id(input, 3)

  summary_textrank_bahasa.append(result)

file_name = 'pengujian_textrank_bahasa.xlsx'

print_test = save_test_data(file_name, reference_summary_bahasa, summary_textrank_bahasa)
print_test

'DataFrame saved to pengujian_textrank_bahasa.xlsx'

In [ ]:
# pegasus rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_bahasa["test"][i]["news_text"]
  result = f_pegasus_id(input)

  summary_pegasus_bahasa.append(result)

file_name = 'pengujian_pegasus_bahasa.xlsx'

In [ ]:
print_test = save_test_data(file_name, reference_summary_bahasa, summary_pegasus_bahasa)
print_test

'DataFrame saved to pengujian_pegasus_bahasa.xlsx'

In [ ]:
# hybrid rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_bahasa["test"][i]["news_text"]
  result = f_hybrid_id(input, 2)

  summary_hybrid_bahasa.append(result)

file_name = 'pengujian_hybrid_bahasa.xlsx'

print_test = save_test_data(file_name, reference_summary_bahasa, summary_hybrid_bahasa)
print_test

'DataFrame saved to pengujian_hybrid_bahasa.xlsx'